# EDA Notebook

## Library

In [1]:
import sys

sys.path.append("..")
sys.path.append("../../inputs")

## Logging

In [3]:
import logging

logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

log = logging.getLogger(__name__)

## Load Data

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Train Data

このデータセットには、何千もの投資案件の実際の履歴データから得られた特徴が含まれています。難読化された指標を予測し、売買の意思決定を行うことが課題です。

このコンペティションは、時系列APIに依存し、モデルが時間を遡ることがないようにするコードコンペティションです。APIを使用するには、評価ページの指示に従います。ノートブックを提出すると、そのノートブックは未見のテストで再実行されます。これは予測コンペでもあり、最終的なプライベートリーダーボードはトレーニング期間終了後に集められたデータを使って決定されるため、パブリックリーダーボードとプライベートリーダーボードの重複はゼロとなります。


In [5]:
%%time

train = pd.read_csv("../../inputs/train.csv")

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3141410 entries, 0 to 3141409
Columns: 304 entries, row_id to f_299
dtypes: float64(301), int64(2), object(1)
memory usage: 7.1+ GB


- `row_id`: 行の一意な識別子。
- `time_id`: データが収集された時刻を表すIDコードです。時間IDは順番に並んでいるが、時間ID間の実時間は一定ではなく、最終的なプライベートテストセットではトレーニングセットより**短くなる**可能性が高い。
- `investment_id`: 投資のIDコードです。すべての投資家がすべての時間帯のIDでデータを持っているわけではありません。
- `target`: 目的変数
- `[f_0:f_299]`: 市場データから生成された匿名化された特徴量。

In [7]:
train

,row_id,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,f_5,...,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,0_1,0,1,-0.300875,0.932573,0.113691,-0.402206,0.378386,-0.203938,-0.413469,...,0.366028,-1.095620,0.200075,0.819155,0.941183,-0.086764,-1.087009,-1.044826,-0.287605,0.321566
1,0_2,0,2,-0.231040,0.810802,-0.514115,0.742368,-0.616673,-0.194255,1.771210,...,-0.154193,0.912726,-0.734579,0.819155,0.941183,-0.387617,-1.087009,-0.929529,-0.974060,-0.343624
2,0_6,0,6,0.568807,0.393974,0.615937,0.567806,-0.607963,0.068883,-1.083155,...,-0.138020,0.912726,-0.551904,-1.220772,-1.060166,-0.219097,-1.087009,-0.612428,-0.113944,0.243608
3,0_7,0,7,-1.064780,-2.343535,-0.011870,1.874606,-0.606346,-0.586827,-0.815737,...,0.382201,0.912726,-0.266359,-1.220772,0.941183,-0.609113,0.104928,-0.783423,1.151730,-0.773309
4,0_8,0,8,-0.531940,0.842057,-0.262993,2.330030,-0.583422,-0.618392,-0.742814,...,-0.170365,0.912726,-0.741355,-1.220772,0.941183,-0.588445,0.104928,0.753279,1.345611,-0.737624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3141405,1219_3768,1219,3768,0.033600,0.093530,-0.720275,-0.345497,-0.438781,-0.166972,-0.437182,...,-0.285908,-1.232434,-0.660579,0.875537,0.421628,-0.428097,-0.075548,-0.533092,-0.193732,-0.581394
3141406,1219_3769,1219,3769,-0.223264,-1.344935,-0.199987,-0.107702,-0.454677,-0.221914,-0.141174,...,0.184517,-1.232434,-0.670493,0.875537,0.421628,-0.729949,-1.514277,0.013145,-0.890270,-0.589705
3141407,1219_3770,1219,3770,-0.559415,0.979489,-1.110491,1.006980,-0.467307,-0.159549,1.355671,...,-0.756332,-1.232434,0.820784,-1.142157,0.421628,-0.363329,1.363181,-0.079106,-1.580124,-0.297625
3141408,1219_3772,1219,3772,0.009599,-2.565332,0.320301,0.076600,1.380182,-0.155366,-0.689000,...,-0.756332,-1.232434,0.133074,-1.142157,0.421628,-0.375288,-1.514277,-0.973762,0.608647,-0.372040


### Example Test Data

投稿時にAPIがどのような形状・形式のデータをあなたのノートブックに配信するかを示すために提供されるランダムデータです。

In [8]:
test = pd.read_csv("../../inputs/example_test.csv")

In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Columns: 303 entries, row_id to f_299
dtypes: float64(300), int64(2), object(1)
memory usage: 21.4+ KB


In [10]:
test

,row_id,time_id,investment_id,f_0,f_1,f_2,f_3,f_4,f_5,f_6,...,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,1220_1,1220,1,0.874944,0.457875,0.962572,0.372061,0.004509,0.971304,0.097477,...,0.514031,0.828523,0.437697,0.810764,0.693963,0.186403,0.569144,0.120386,0.419603,0.936966
1,1220_2,1220,2,0.554862,0.445478,0.714034,0.891988,0.703331,0.264478,0.600636,...,0.620919,0.301540,0.687464,0.400168,0.908198,0.272901,0.222430,0.748324,0.858802,0.362829
2,1221_0,1221,0,0.175070,0.918464,0.044379,0.505868,0.037160,0.169001,0.211365,...,0.397710,0.789382,0.238078,0.254068,0.795084,0.491928,0.376725,0.079489,0.995041,0.482283
3,1221_1,1221,1,0.355239,0.689773,0.452643,0.134863,0.794958,0.837197,0.036773,...,0.893571,0.322879,0.674692,0.114160,0.818483,0.287065,0.710537,0.550434,0.899869,0.928997
4,1221_2,1221,2,0.912778,0.462684,0.454067,0.491009,0.050547,0.250051,0.686295,...,0.031662,0.703427,0.861476,0.897927,0.659051,0.356148,0.375990,0.209798,0.175153,0.727160
5,1222_0,1222,0,0.678116,0.158996,0.058148,0.253677,0.984982,0.323493,0.307990,...,0.439604,0.487491,0.080697,0.539750,0.246645,0.435054,0.436709,0.315042,0.133657,0.731483
6,1222_1,1222,1,0.769912,0.060028,0.676047,0.853550,0.411155,0.747515,0.649063,...,0.419152,0.659254,0.198072,0.655397,0.428209,0.769999,0.960219,0.734675,0.028576,0.320976
7,1222_2,1222,2,0.738876,0.980038,0.696796,0.591012,0.882052,0.311484,0.729229,...,0.341144,0.758328,0.010728,0.276859,0.979913,0.486605,0.670140,0.515824,0.580929,0.139308
8,1223_0,1223,0,0.281445,0.571472,0.986828,0.021403,0.789577,0.005765,0.409379,...,0.357828,0.000019,0.504530,0.169174,0.216349,0.503643,0.052721,0.795965,0.909602,0.291668


### Example Sample Submission

API の一般公開されたコピーが正しいデータの形と形式を提供するように提供される提出ファイルの例です。

In [11]:
sample_sub = pd.read_csv("../../inputs/example_sample_submission.csv")

In [12]:
sample_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   row_id   9 non-null      object
 1   time_id  9 non-null      int64 
 2   target   9 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 344.0+ bytes


In [13]:
sample_sub

,row_id,time_id,target
0,1220_1,1220,0
1,1220_2,1220,0
2,1221_0,1221,0
3,1221_1,1221,0
4,1221_2,1221,0
5,1222_0,1222,0
6,1222_1,1222,0
7,1222_2,1222,0
8,1223_0,1223,0


### Time Series API

時系列APIの詳細 - APIはバッチごとにデータを提供し、バッチごとに1つの`time_id`を持つ。

- テストセットには約100万行が含まれると予想される
- APIは、初期化後におよそ0.25GBのメモリを必要とします。初期化ステップ(env.iter_test())では、それ以上のメモリが必要になります。
- また、APIの実行時間は、データの読み込みと配信に15分未満を予定しています。

In [2]:
import ubiquant

env = ubiquant.make_env()  # initialize the environment
iter_test = env.iter_test()  # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df["target"] = 0  # make your predictions here
    env.predict(sample_prediction_df)  # register your predictions

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


### Evaluation

提出物は、各タイムIDのピアソン相関係数の平均値で評価されます。


Pearsonの(積率)相関係数とは
https://www.st-hakky-blog.com/entry/2018/01/30/004659

以下の式で表され、-1～1の値の間で値が変化します。

![image](https://user-images.githubusercontent.com/1638500/150124817-59c1f5ad-acd0-424e-9f39-56f6ccda2784.png)

変数間の直線関係の強度を知りたいときにこの相関係数を使用します。「データにおいて、xとyのばらつきぐあいが完全に一致していたら、相関係数が1になる」ことがわかります。

また、ピアソンの相関係数では、変数が正規分布に従っていることを仮定しています。なので、正規分布に従っていると仮定できるデータがあったとして、そのデータの直線関係の強度を知りたい時、これは使えます。



### Timeline

このコンペティションは、アクティブなトレーニング期間と、実際の市場データに対してモデルを実行する第2期間を設けた予測コンペティションです。

最終提出期限以降、選択されたノートブックに対して実行されるデータの更新を反映して、リーダーボードが定期的に更新されます。更新はおよそ2週間おきに行われる予定です。

- 学習期間締め切り: 4/19 9:00
- 評価期間終了: 7/18

## EDA